In [1]:
import os
import shutil
from subprocess import Popen, PIPE
from multiprocessing import Pool, cpu_count

import numpy as np

In [2]:
def run_docker(cmd, container_name):  
    try:
        proc = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
        _ = proc.wait()
        if _!=0:
            print(proc.stdout())
            return False
    except Exception as e:
        print (proc.communicate())
        print (e)
        proc_kill = Popen('docker stop ' + container_name, shell=True, stdout=PIPE)
        proc_kill.wait()
        proc_kill = Popen('docker rm ' + container_name, shell=True, stdout=PIPE)
        proc_kill.wait()
        proc.kill()
        return False
    return True

In [3]:
def bet_brain_mask(docker_container_name, path_to_sourse, file_name):
    DOCKER_NAME = 'brain_pipeline/frsrf_minc_fsl'
    return '''docker run --rm \
    --name {1} \
    -v {2}:/input \
    {0} \
    bash -c ". /etc/fsl/5.0/fsl.sh \
    && bet /input/{3} /input/brainmask_{3} -m -n -f .4\
    " '''.format(DOCKER_NAME, # 0
                 docker_container_name, # 1
                 path_to_sourse, # 2
                 file_name # 3
                 )

In [4]:
def skull_strip_for(path_to_sourse, file_name):
    '''
    For new registered img apply skull stripping.
    '''
    docker_container_name = get_rnd_string()
    cmd = bet_brain_mask(docker_container_name, path_to_sourse, file_name)
    print(cmd)
    _ = run_docker(cmd, docker_container_name)
    return _

In [5]:
def flirt_brain_isotropy(docker_container_name, path_to_sourse):
    DOCKER_NAME = 'brain_pipeline/frsrf_minc_fsl'
    return '''docker run --rm \
    --name {1} \
    -v {2}:/input \
    {0} \
    bash -c ". /etc/fsl/5.0/fsl.sh \
    && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii.gz -omat /input/pre/some_transform.mat -applyisoxfm 1 \
    && flirt -in /input/wmh.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/reg_wmh.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1 \
    && flirt -in /input/pre/T1.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/pre/reg_T1.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1 \
    " '''.format(DOCKER_NAME, # 0
                 docker_container_name, # 1
                 path_to_sourse # 2
                 )

In [6]:
def isotr_transform_for(path_to_sourse):
    '''
    For img apply tranform.
    '''
    docker_container_name = get_rnd_string()
    cmd = flirt_brain_isotropy(docker_container_name, path_to_sourse)
    print(cmd)
    _ = run_docker(cmd, docker_container_name)
    return _

In [7]:
def get_rnd_string():
    import string
    import random
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))

In [8]:
#
# #
# # # just copy all tree to new directory 
# #
#

path_to_copy = '/mount/export/RESAMPLED_MICCAI_WMHS/' # /nmt/x05-ssd/RESAMPLED_MICCAI_WMHS/
path_to_orig = '/mount/export/MICCAI_WMHS/'

for path, dirs, files in os.walk(path_to_orig):
    for f in files:
        if ('wmh.nii.gz' == f or 
            ('pre' in path and ('T1.nii.gz' == f or 'FLAIR.nii.gz' == f))):
            val = 3 if 'wmh.nii.gz' == f else 4
            path_to_new_files = os.path.join(path_to_copy, '/'.join(path.split('/')[-val:]))
            if not os.path.exists(path_to_new_files):
                os.makedirs(path_to_new_files, exist_ok=True)
            print ('copy', path+'/'+f, ' _to_ ', path_to_new_files + '/' + f)
            shutil.copy(os.path.join(path, f), os.path.join(path_to_new_files, f))

copy /mount/export/MICCAI_WMHS/Amst/GE3T/114/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/114/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/114/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/114/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/114/pre/FLAIR.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/114/pre/FLAIR.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/100/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/100/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/100/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/100/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/100/pre/FLAIR.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/100/pre/FLAIR.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/107/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/107/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Amst/GE3T/107/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI

copy /mount/export/MICCAI_WMHS/Utr/Utrecht/21/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/21/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/21/pre/FLAIR.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/21/pre/FLAIR.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/6/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/6/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/6/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/6/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/6/pre/FLAIR.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/6/pre/FLAIR.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/29/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/29/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/29/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/29/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Utr/Utrecht/29/pre/FLAIR.nii.gz  _to_  /mount/expor

copy /mount/export/MICCAI_WMHS/Sing/Singapore/57/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/57/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Singapore/57/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/57/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Singapore/57/pre/FLAIR.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/57/pre/FLAIR.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Singapore/69/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/69/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Singapore/69/pre/T1.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/69/pre/T1.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Singapore/69/pre/FLAIR.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/69/pre/FLAIR.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Singapore/67/wmh.nii.gz  _to_  /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/67/wmh.nii.gz
copy /mount/export/MICCAI_WMHS/Sing/Sing

In [9]:
# and then running flirt brain anisotropy

for path, dirs, files in os.walk(path_to_copy):
    if 'wmh.nii.gz' in files:
        path_to_new_files = os.path.join(path_to_copy, '/'.join(path.split('/')[-3:]))
        print (path_to_new_files)
        isotr_transform_for(path_to_new_files)

/mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/114
docker run --rm     --name 7XPXS73DJP     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/114:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii.gz -omat /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/wmh.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/reg_wmh.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/pre/T1.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/pre/reg_T1.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     " 
/mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/100
docker run --rm     --name R0F0JVDKL0     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/100:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii

/mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/110
docker run --rm     --name 7WZV5VPYH5     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/110:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii.gz -omat /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/wmh.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/reg_wmh.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/pre/T1.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/pre/reg_T1.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     " 
/mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/137
docker run --rm     --name G3JG6EZ805     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/137:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii

/mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/35
docker run --rm     --name G72Y8WN3PF     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/35:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii.gz -omat /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/wmh.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/reg_wmh.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/pre/T1.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/pre/reg_T1.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     " 
/mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/21
docker run --rm     --name XZGK51HQZH     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/21:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR

/mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/33
docker run --rm     --name KARQN752ZY     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/33:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii.gz -omat /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/wmh.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/reg_wmh.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/pre/T1.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/pre/reg_T1.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     " 
/mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/61
docker run --rm     --name KNFECDI5ZX     -v /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/61:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg

/mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/66
docker run --rm     --name M1NNG6O9M8     -v /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/66:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/pre/reg_FLAIR.nii.gz -omat /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/wmh.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/reg_wmh.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     && flirt -in /input/pre/T1.nii.gz -ref /input/pre/reg_FLAIR.nii.gz -out /input/pre/reg_T1.nii.gz -init /input/pre/some_transform.mat -applyisoxfm 1     " 
/mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/58
docker run --rm     --name H8WP9P9IO5     -v /mount/export/RESAMPLED_MICCAI_WMHS/Sing/Singapore/58:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && flirt -in /input/pre/FLAIR.nii.gz -ref /input/pre/FLAIR.nii.gz -out /input/p

In [10]:
# also apply skull stripping 

for path, dirs, files in os.walk(path_to_copy):
    for f in files:
        if f == 'reg_T1.nii.gz':
            skull_strip_for(path, f)

docker run --rm     --name FSO1XVFVJP     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/114/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.nii.gz -m -n -f .4    " 
docker run --rm     --name 4DOXV1FYV9     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/100/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.nii.gz -m -n -f .4    " 
docker run --rm     --name SCSOQA1QOD     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/107/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.nii.gz -m -n -f .4    " 
docker run --rm     --name YC035TCF6I     -v /mount/export/RESAMPLED_MICCAI_WMHS/Amst/GE3T/102/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.n

docker run --rm     --name ICS9G2LS7R     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/17/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.nii.gz -m -n -f .4    " 
docker run --rm     --name 051681Q9PU     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/11/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.nii.gz -m -n -f .4    " 
docker run --rm     --name T0G1Z0BNTX     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/27/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_T1.nii.gz -m -n -f .4    " 
docker run --rm     --name ESZOO7L0C3     -v /mount/export/RESAMPLED_MICCAI_WMHS/Utr/Utrecht/23/pre:/input     brain_pipeline/frsrf_minc_fsl     bash -c ". /etc/fsl/5.0/fsl.sh     && bet /input/reg_T1.nii.gz /input/brainmask_reg_

In [23]:
!ls -R /nmnt/x05-ssd/RESAMPLED_MICCAI_WMHS/Amst/GE3T/126/

/nmnt/x05-ssd/RESAMPLED_MICCAI_WMHS/Amst/GE3T/126/:
pre  reg_wmh.nii.gz  wmh.nii.gz

/nmnt/x05-ssd/RESAMPLED_MICCAI_WMHS/Amst/GE3T/126/pre:
brainmask_reg_T1_mask.nii.gz  reg_FLAIR.nii.gz	some_transform.mat
FLAIR.nii.gz		      reg_T1.nii.gz	T1.nii.gz
